In [ ]:
# -- adding custom endpoints that use custom pools
# -- mock vs private
# -- context.node
# -- settings and state
# -- service account via settings or k8s labels
# -- how to clear state
# -- how to implement a basic rate limiter using context.user
# -- creating a big query endpoint with a try catch
# -- how to delete / replace api endpoints
# -- testing endpoints and their mock / private versions
# -- executing a large query and checking blob storage

In [ ]:
NODE_URL = "http://<node-url>
ADMIN_EMAIL = "info@openmined.org"
ADMIN_PASSWORD = "changethis"
BQ_CREDENTIALS = '....'

In [ ]:
client = sy.login(url=NODE_URL, email=ADMIN_EMAIL, password=ADMIN_PASSWORD)

## Create a new TwinAPIEndpoint

In [ ]:
# Mock API
@sy.api_endpoint_method(
    settings={
         "CREDENTIALS": BQ_CREDENTIALS,       
    }
)
def mock_query_function(
    context,
    sql_query: str,  
) -> str:
    # third party
    import pandas as pd
    import numpy as np
    
    # Set the seed for reproducibility
    np.random.seed(42)
    try:

        # Generate mock data
        data = {
            "Name": [f"Name_{i}" for i in range(1, 11)],
            "Age": np.random.randint(20, 50, size=10),
            "Email": [f"name_{i}@example.com" for i in range(1, 11)],
            "JoinDate": pd.date_range(start="2023-01-01", periods=10, freq='M').strftime('%Y-%m-%d').tolist(),
            "Salary": np.random.randint(40000, 120000, size=10)
        }
        
        # Create DataFrame
        return pd.DataFrame(data)
    except Exception as e:
        return SyftError(message="Ops! Something went wrong. please, contact your admin")


# Private API
@sy.api_endpoint_method(
    settings={
         "CREDENTIALS": BQ_CREDENTIALS,       
    }
)
def private_query_function(
    context,
    sql_query: str,  
) -> str:
    # third party
    from google.cloud import bigquery
    from syft.service.response import SyftError

    # Set the seed for reproducibility
    np.random.seed(42)
    try:

        # Client query
        client = bigquery.Client()

        # Generate mock data
        rows = client.query_and_wait(
            sql_query,
            project=project,
        )
        
        # Create DataFrame
        return rows
    except Exception as e:
        return SyftError(message="Ops! Something went wrong. please, contact your admin")



# Create new Twin API using bigquery-pool as a worker pool
new_endpoint = sy.TwinAPIEndpoint(
    path="new_api.ask",
    description="Ask SQL Queries using our BQ",
    private_function=private_query_function,
    mock_function=mock_query_function,
    worker_pool="bigquery-pool"
)

client.custom_api.add(endpoint=new_endpoint)

## How to List Twin APIS

In [ ]:
client.service.api.api_endpoints()

## How to Update Twin APIS

In [ ]:
@sy.api_endpoint_method()
def new_mock_function(
    context,
    sql_query: str,  
) -> str:
    return "New Mock Function"

@sy.api_endpoint_method()
def new_private_function(
    context,
    sql_query: str,  
) -> str:
    return "New Mock Function"


client.services.api.update(
        endpoint_path: "new_api.ask",
        mock_function: new_mock_function, # Optional
        private_function: new_private_function, # Optional
        hide_mock_definition: True, # Optional
        endpoint_timeout: 1000, # Optional
)

## How to delete API endpoints

In [ ]:
client.services.api.delete(endpoint_path="new_api.ask")